In [23]:
import glob
import os
from pymatgen.io.vasp.outputs import Vasprun
import math
import numpy
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

E:\anaconda64bits\lib\site-packages\pymatgen\__init__.py:34: UserWarning: With effect from pmg 5.0, all pymatgen settings are prefixed with a "PMG_". E.g., "PMG_VASP_PSP_DIR" instead of "VASP_PSP_DIR".
  warnings.warn('With effect from pmg 5.0, all pymatgen settings are'


In [24]:
#Suppose we are in the "Lix(M)O2" folder, with multiple folders of possible swap situations.
#The analysis process is seperated to the following part:
# -- find the relaxed original structure and record the energy before swapping.
# -- check if the swapped structure has converged or not.
# -- if so, calculate the swap energy.
# -- normalize the swap energy and calculate the mixing temperature.
# -- plot all data on the figure. Phase diagram built.

In [25]:
#Limitations:
#The applied RSM is only valid for LixMO2 system.

In [26]:
"""Find perfect structure's converged energy."""
def find_original_energy():
    #Get the names of sub-folders in the main folder.
    folders = glob.glob('./*')
    files = glob.glob('./*.py')
    dirs = [x for x in folders if x not in files]
    i = 0
    #Go through all sub-folders to find the perfect structure.
    for folder in dirs:
        #go to the sub-folder
        os.chdir(folder)
        v = Vasprun("vasprun.xml")
        #check if it is the perfect structure, if so, grab the energy(here we suppose only the perfect structure's folder contains the key word 'origin')
        if folder.find('origin') > 0:
            initial_energy = v.final_energy
            structure = v.final_structure
            a = i
        #the index of this folder
        i = i + 1
        #go back to the parent dir
        os.chdir(os.path.pardir)
        #remove this name from folder list
        dirs.pop(i)
    return initial_energy,dirs,structure

"""Grab other final energies and substract perfect energy to find out swap energy."""
def record_swap_energy(initial_energy, folders):
    swap_energy_list = []
    for folder in folders:
        os.chdir(folder)
        #check if the structure is converged. If so, grab the final energy and calculate the swap energy.
        v = Vasprun("vasprun.xml")
        if v.converged:
            swap_energy = v.final_energy - initial_energy
            swap_energy_list.append(swap_energy)
        os.chdir(os.path.pardir)
    print(swap_energy_list)
    return swap_energy_list

"""Calculate the mixing temperature for this material. Right now only for Li(M)O2 system.
   The input structure is the original structure."""
def cal_mix_temp(swap_energy_list, structure, dic_outcome):
    #normalize the swap energy.
    indice_O = structure.indices_from_symbol('O')
    indice_Li = structure.indices_from_symbol('Li')
    Num_of_O = len(indice_O)/2
    Num_of_Li = len(indice_Li)
    frac_Li = Num_of_Li/(Num_of_O)
    factor = 1.6*(math.pow(10,-19))*6.02*(math.pow(10,23))/Num_of_O
    H = [energy*factor for energy in swap_energy_list]
    print(H)
    #entropy
    specie_list = structure.types_of_specie
    specie_list.pop(-1)
    total_number = 0
    number_count = []
    for specie in specie_list:
        spe_name = str(specie)
        indice = structure.indices_from_symbol(spe_name)
        num_specie = len(indice)
        number_count.append(num_specie)
        total_number = total_number + num_specie
    fraction = [n/total for n in number_count]
    entropy = 0
    for f in fraction:
        new_entropy = f*(math.log(f,math.e))
        entropy = entropy + new_entropy
    
    #calculate mixing temperature
    T = []
    for h in H:
        t = -h/(entropy*8.31)
        T.append(t)
    T = sorted(T)
    
    #store the outcome in the dictionary
    dic_outcome[frac_Li] = T
    return dict_outcome

In [27]:
#Once we get the data for all Li concentration, we can make a plot
"""Plot the phase diagram."""
def plot_phase_diagram(dict_outcome,filename):
    concentration_list = list(dict_outcome)
    lowest_temp_list = []
    #plot the scattered points
    for con in concentration_list:
        temp_list = dict_outcome[con]
        lowest_temp_list.append(temp_list[0])
        for temp in temp_list:
            temp_float = float(temp)
            plt.scatter(con,temp_float,color='b')
    #plot the mixing boundary.
    plt.plot(concentration_list,lowest_temp_list,color='r')
    plt.xlabel(filename)
    plt.ylabel('Temperature(K)')
    plt.savefig("Mixing Temperature")
 

In [29]:
# Integrated.
initial_energy, dirs, structure = find_original_energy()
swap_energy_list = record_swap_energy(initial_energy, dirs)
dict_outcome = {}
dict_outcome = cal_mix_temp(swap_energy_list, structure, dict_outcome) # the structure is the perfect structure.
filename =  'Lix(Co0.1Ni0.9)O2' # This is the target material name.
plot_phase_diagram(dict_outcome,filename)

NotADirectoryError: [WinError 267] The directory name is invalid: '.\\general.ipynb'